In [ ]:
import asyncio
import aiohttp
from datasets import load_dataset

import vertexai
from vertexai.generative_models import GenerativeModel
import vertexai.preview.generative_models as generative_models

seed =42

# TODO:
- enforce output format for gemini
- add evaluation if there is a golden set for individual model + aggregate
- add for multi-class/ner
- add images
- llama, gpt, claude

In [1]:
from utils import main
from datasets import load_dataset

seed =42

/usr/local/google/home/amirimani/Desktop/projects/LLMTest_NeedleInAHaystack/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gemini_prompt_template = """
<data_description>
{description}
</data_description>
-----------

<context>
{datapoint}
</context>
------------

<labels>
{labels}
</labels>
------------

INSTRUCTION:
- familirize yourself with the data using data_description
- read the context carefully. this is the data point you need to label.
- take your time and label the dadatapoint with the most appropriate option using the provided labels.
- return the result as a single label from the <labels>. Don't provide explanations
"""

In [3]:
dataset = load_dataset("yelp_polarity", split="train") # https://huggingface.co/datasets/yelp_polarity

# take a small sample for dev purposes
dataset_sample = dataset.shuffle(seed=seed).select(range(1000))

# user provided data description
DESCRIPTION = """
This is a dataset for binary sentiment classification.
It contains highly polar yelp reviews.
Negative polarity is class 0, and positive class 1.
"""

LABEL_SET = [0, 1] 

In [7]:
prompt = [gemini_prompt_template.format(description= DESCRIPTION,
                                        datapoint=x,
                                        labels=LABEL_SET) for x in dataset_sample["text"][:5]]
len(prompt)

50

In [8]:
await main(prompt, model="gemini")

['1', '0', '[0]', '1', '1', '1', '1', '0', '0', '[0]', '0', '1 \n', '1', '0', '0', '## 0', '0', '1', '## 0 \n', '0', '0', '1', '1', '## Label: 0\n\n**Explanation:**\n\nThe review expresses negative sentiment towards the restaurant. The customer had a bad experience due to long waiting time and attributes it to the new management. They state they will not return, indicating dissatisfaction. \n', '0 \n', '1', '0', '[1] \n', '1', '1', '1', '1 \n', '0', '## 1', '[1]', '1', '1', '1', '0', '1', '0\n', '[1] \n', '1', '0', '1', '0 \n', '1', '1 \n', '## Data point review:\n\nThis review appears extremely positive, with exclamation marks and all-caps emphasizing enthusiasm. Phrases like "filled with the Holy Spirit" and "AMAZING" further suggest a strong positive sentiment. \n\n## Label:\n\nTherefore, the appropriate label for this review is **1**. \n', '[0]']
